<div style="background-color:#4EA72E;  padding:5px; border-radius:20px;">
    
  <p style="color:#DC2512; text-align:center; font-size:40px;"> E-commerce Data Warehouse Project  </p>
 
</div>

<div style="background-color:#17DCD7;  padding:10px; border-radius:10px;">
    
  <p style="color:#D67218; text-align:center; font-size:20px;"> Import Library  </p>
 
</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


<div style="background-color:#17DCD7;  padding:10px; border-radius:10px;">
    
  <p style="color:#D67218; text-align:center; font-size:20px;"> Insert Data  </p>
 
</div>

In [ ]:
customers=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_customers_dataset.csv')

geo=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_geolocation_dataset.csv')
order_items=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_order_items_dataset.csv')
order_payments=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_order_payments_dataset.csv')
order_reviews=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_order_reviews_dataset.csv')
order_dataset=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_orders_dataset.csv')
products=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_products_dataset.csv')
sellers=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/olist_sellers_dataset.csv')
category_translation=pd.read_csv('E:/Data Analysis/Projct3/Main_project/ecommerce_data_engineering/Raw_data/product_category_name_translation.csv')


In [ ]:
Q1 = order_items['price'].quantile(0.25)
Q3 = order_items['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = order_items[(order_items['price'] < lower_bound) | (order_items['price'] > upper_bound)]
print(outliers['price'])
print("outlier_price:",outliers['price'].value_counts().sum())

7         810.00
16        639.00
26        289.00
30        397.00
39        809.10
           ...  
112586    299.99
112592    278.00
112632    736.00
112645    299.99
112646    350.00
Name: price, Length: 8427, dtype: float64
outlier_price: 8427


In [ ]:
Q99 = order_items['freight_value'].quantile(.99)
outliers = order_items[(order_items['freight_value'] >= Q99)]
print(outliers['freight_value'])
print("Q99:",  Q99)
print("outlier_freight_value99:",outliers['freight_value'].value_counts().sum())
order_items=order_items[order_items['freight_value']<Q99]
print(order_items['freight_value'])

124        85.97
162        87.28
352        97.56
456        98.13
590        94.98
           ...  
112014     86.77
112233    193.21
112359    170.11
112450     84.52
112472    127.55
Name: freight_value, Length: 1128, dtype: float64
Q99: 84.52
outlier_freight_value99: 1128
0         13.29
1         19.93
2         17.87
3         12.79
4         18.14
          ...  
112645    43.41
112646    36.53
112647    16.95
112648     8.72
112649    12.79
Name: freight_value, Length: 111522, dtype: float64


In [ ]:
order_items[order_items['freight_value']<0].value_counts().sum()

0

<div style="background-color:#92D050;  padding:10px; border-radius:5px;">
    
  <p style="color:#EB2213; text-align:center; font-size:15px;"> Cleaning </p>
 
</div>

In [ ]:

filtered_df = order_items[order_items['freight_value']>=Q99]
order_items = order_items.drop(index=filtered_df.index)
order_items['shipping_limit_date']=pd.to_datetime(order_items['shipping_limit_date'])


<div style="background-color:#17DCD7;  padding:10px; border-radius:10px;">
    
  <p style="color:#D67218; text-align:center; font-size:20px;"> order_reviews</p>
 
</div>

<div style="background-color:#92D050;  padding:10px; border-radius:5px;">
    
  <p style="color:#EB2213; text-align:center; font-size:15px;"> Cleaning </p>
 
</div>

In [ ]:
order_reviews['review_comment_title']=order_reviews['review_comment_title'].fillna('NoTitle')
order_reviews['review_comment_message']=order_reviews['review_comment_message'].fillna('Nomessage')

In [ ]:
order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NoTitle,Nomessage,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NoTitle,Nomessage,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NoTitle,Nomessage,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NoTitle,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NoTitle,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


<div style="background-color:#17DCD7;  padding:10px; border-radius:10px;">
    
  <p style="color:#D67218; text-align:center; font-size:20px;"> order_dataset</p>
 
</div>

<div style="background-color:#92D050;  padding:10px; border-radius:5px;">
    
  <p style="color:#EB2213; text-align:center; font-size:15px;"> Cleaning </p>
 
</div>

In [ ]:
order_dataset['order_purchase_timestamp']=pd.to_datetime(order_dataset['order_purchase_timestamp'])
order_dataset['order_approved_at']=pd.to_datetime(order_dataset['order_approved_at'])
order_dataset['order_delivered_carrier_date']=pd.to_datetime(order_dataset['order_delivered_carrier_date'])
order_dataset['order_delivered_customer_date']=pd.to_datetime(order_dataset['order_delivered_customer_date'])
order_dataset['order_estimated_delivery_date']=pd.to_datetime(order_dataset['order_estimated_delivery_date'])

In [ ]:

filtered_order_dataset = order_dataset[order_dataset['order_delivered_carrier_date']>order_dataset['order_delivered_customer_date']]
order_dataset = order_dataset.drop(index=filtered_order_dataset.index)


In [ ]:
order_dataset=order_dataset.dropna(subset=['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date'])

<div style="background-color:#17DCD7;  padding:10px; border-radius:10px;">
    
  <p style="color:#D67218; text-align:center; font-size:20px;"> products</p>
 
</div>

<div style="background-color:#92D050;  padding:10px; border-radius:5px;">
    
  <p style="color:#EB2213; text-align:center; font-size:15px;"> Cleaning </p>
 
</div>

In [ ]:
products=products.dropna(subset=['product_category_name', 'product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g'])

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32340 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32340 non-null  object 
 1   product_category_name       32340 non-null  object 
 2   product_name_lenght         32340 non-null  float64
 3   product_description_lenght  32340 non-null  float64
 4   product_photos_qty          32340 non-null  float64
 5   product_weight_g            32340 non-null  float64
 6   product_length_cm           32340 non-null  float64
 7   product_height_cm           32340 non-null  float64
 8   product_width_cm            32340 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.5+ MB


<div style="background-color:#37B3B3;  padding:10px; border-radius:5px;">
    
  <p style="color:#D67218; text-align:center; font-size:15px;"> Save File </p>
 
</div>

In [ ]:
geo.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Geo_clean.csv", index=False)

In [ ]:
customers.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Customers_clean.csv", index=False)

In [ ]:
order_items.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Order_items_clean.csv", index=False)

In [ ]:
order_payments.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Order_payments_clean.csv", index=False)

In [ ]:
order_reviews.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Order_reviews_clean.csv", index=False)

In [ ]:
order_dataset.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Order_dataset_clean.csv", index=False)

In [ ]:
products.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Products_clean.csv", index=False)

In [ ]:
sellers.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Sellers_clean.csv", index=False)

In [ ]:
category_translation.to_csv("/Users/Ahmad/ecommerce_data_engineering/Clean_data/Category_translation_clean.csv", index=False)

<div style="background-color:#D96211;  padding:10px; border-radius:10px;">
    
  <p style="color:#0CE307; text-align:center; font-size:20px;"> Model Data </p>
 
</div>

In [ ]:
order1=pd.merge(order_items,order_payments[['order_id', 'payment_value']], on='order_id', how='left')

In [ ]:
order2=pd.merge(order1,order_reviews[['order_id','review_score']], on='order_id', how='left')

In [ ]:
customer_dim=pd.merge(order_dataset, customers, on='customer_id', how='right')

In [ ]:
seller_dim=sellers

In [ ]:
order_fact=pd.merge(order2,customer_dim[['order_id','customer_id']], on='order_id', how='left')

In [ ]:
order_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117098 entries, 0 to 117097
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             117098 non-null  object        
 1   order_item_id        117098 non-null  int64         
 2   product_id           117098 non-null  object        
 3   seller_id            117098 non-null  object        
 4   shipping_limit_date  117098 non-null  datetime64[ns]
 5   price                117098 non-null  float64       
 6   freight_value        117098 non-null  float64       
 7   payment_value        117095 non-null  float64       
 8   review_score         116135 non-null  float64       
 9   customer_id          114489 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 8.9+ MB


In [ ]:
product_dim=pd.merge(products,category_translation, on='product_category_name', how='inner')

In [ ]:

order_fact.to_csv("/Users/Ahmad/cleaning/order_fact.csv", index=False)

customer_dim.to_csv("/Users/Ahmad/cleaning/customer_dim.csv", index=False)

seller_dim.to_csv("/Users/Ahmad/cleaning/seller_dim.csv", index=False)

product_dim.to_csv("/Users/Ahmad/cleaning/product_dim.csv", index=False)

<div style="background-color:#647CD6;  padding:10px; border-radius:10px;">
    
  <p style="color:#0CE307; text-align:center; font-size:20px;"> Validation </p>
 
</div>

In [ ]:
 assert order_fact["order_id"].isin(order_items["order_id"]).all()

In [ ]:
missing_products = order_fact[~order_fact["product_id"].isin(product_dim["product_id"])]


order_fact_filtered = order_fact[order_fact["product_id"].isin(product_dim["product_id"])]

order_fact=order_fact_filtered

In [ ]:
 assert order_fact["product_id"].isin(product_dim["product_id"]).all()

In [ ]:
 assert order_fact["seller_id"].isin(seller_dim["seller_id"]).all()

In [ ]:
missing_customers = order_fact[~order_fact["order_id"].isin(customer_dim["order_id"])]
#print(f"Products missing from product_dim: {missing_products['product_id'].unique()}")

order_fact_filtered = order_fact[order_fact["order_id"].isin(customer_dim["order_id"])]

order_fact=order_fact_filtered

In [ ]:
 assert order_fact["order_id"].isin(customer_dim["order_id"]).all()

In [ ]:
assert order_fact["customer_id"].isin(customer_dim["customer_id"]).all()

<div style="background-color:#56E496;  padding:10px; border-radius:10px;">
    
  <p style="color:#D02E0E; text-align:center; font-size:15px;"> Outlier Of Model </p>
 
</div>

In [ ]:
print("freight_valu_negative:", (order_fact['freight_value']<0).value_counts())
print("order_fact_null:", order_fact.isnull().sum())
order_fact_dupli=order_fact[order_fact.duplicated()]
print("order_fact_dupli:",len(order_fact_dupli))

freight_valu_negative: freight_value
False    112853
Name: count, dtype: int64
order_fact_null: order_id                 0
order_item_id            0
product_id               0
seller_id                0
shipping_limit_date      0
price                    0
freight_value            0
payment_value            3
review_score           834
customer_id              0
dtype: int64
order_fact_dupli: 1032


In [ ]:
print("product_dim_null:", product_dim.isnull().sum())
product_dim_dupli=product_dim[product_dim.duplicated]
print("product_dim_dupli:", product_dim_dupli.value_counts().sum())

product_dim_null: product_id                       0
product_category_name            0
product_name_lenght              0
product_description_lenght       0
product_photos_qty               0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
product_category_name_english    0
dtype: int64
product_dim_dupli: 0


In [ ]:
print("seller_dim_null:", seller_dim.isnull().sum())
seller_dim_dupli=seller_dim[seller_dim.duplicated]
print("seller_dim_dupli:",seller_dim_dupli.value_counts().sum())

seller_dim_null: seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64
seller_dim_dupli: 0


In [ ]:
print("customer_dim_null:", seller_dim.isnull().sum())
customer_dim_dupli=customer_dim[customer_dim.duplicated]
print(customer_dim_dupli.value_counts().sum())

customer_dim_null: seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64
0
